<a href="https://colab.research.google.com/github/Acor-Kyudou/Neuro_Motion/blob/main/Test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne torch pytorch-lightning torchmetrics onnxruntime opencv-python matplotlib numpy seaborn scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
import mne
import os
import numpy as np
from scipy import signal
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
import onnxruntime as ort
import matplotlib.pyplot as plt
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score
from sklearn.metrics import confusion_matrix, roc_curve, auc
import seaborn as sns
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
%matplotlib inline
plt.rcParams['axes.facecolor'] = 'lightgray'

Mounted at /content/drive


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/dataset/raw_data/files"
MODEL_DIR = "/content/drive/MyDrive/dataset/EEGMotorImagery-master"
OPEN_CLOSE_LEFT_RIGHT_FIST = [3, 7, 11]
CLASSES = ["left", "right"]

# Check dataset path
if not os.path.exists(DATASET_PATH):
    raise ValueError(f"Dataset path does not exist: {DATASET_PATH}")
print(f"Dataset path exists: {DATASET_PATH}")

Dataset path exists: /content/drive/MyDrive/dataset/raw_data/files


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, num_hiddens, dropout, max_len=1000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.p = torch.zeros((1, max_len, num_hiddens))
        x = torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1) / torch.pow(10000, torch.arange(
            0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        self.p[:, :, 0::2] = torch.sin(x)
        self.p[:, :, 1::2] = torch.cos(x)

    def forward(self, x):
        x = x + self.p[:, :x.shape[1], :].to(x.device)
        return self.dropout(x)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, dim_feedforward, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, dim_feedforward),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, embed_dim),
        )
        self.layernorm0 = nn.LayerNorm(embed_dim)
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.dropout = dropout

    def forward(self, x):
        y, att = self.attention(x, x, x)
        y = nn.functional.dropout(y, self.dropout, training=self.training)
        x = self.layernorm0(x + y)
        y = self.mlp(x)
        y = nn.functional.dropout(y, self.dropout, training=self.training)
        x = self.layernorm1(x + y)
        return x

In [ ]:
class EEGClassificationModel(nn.Module):
    def __init__(self, eeg_channel, dropout=0.1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(eeg_channel, eeg_channel, 11, 1, padding=5, bias=False),
            nn.BatchNorm1d(eeg_channel),
            nn.ReLU(True),
            nn.Dropout1d(dropout),
            nn.Conv1d(eeg_channel, eeg_channel * 2, 11, 1, padding=5, bias=False),
            nn.BatchNorm1d(eeg_channel * 2),
        )
        self.transformer = nn.Sequential(
            PositionalEncoding(eeg_channel * 2, dropout),
            TransformerBlock(eeg_channel * 2, 4, eeg_channel // 8, dropout),
            TransformerBlock(eeg_channel * 2, 4, eeg_channel // 8, dropout),
        )
        self.mlp = nn.Sequential(
            nn.Linear(eeg_channel * 2, eeg_channel // 2),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(eeg_channel // 2, 1),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.permute(0, 2, 1)
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x = x.mean(dim=-1)
        x = self.mlp(x)
        return x

In [ ]:
print("Loading test EDF files...")
def get_edf_paths(subject_ids, run_numbers):
    physionet_paths = []
    for subject_id in subject_ids:
        subject_folder = f"S{subject_id:03d}"
        subject_path = os.path.join(DATASET_PATH, subject_folder)
        if not os.path.exists(subject_path):
            print(f"Subject path does not exist: {subject_path}")
            continue
        for run in run_numbers:
            run_file = f"{subject_folder}R{run:02d}.edf"
            file_path = os.path.join(subject_path, run_file)
            if os.path.exists(file_path):
                physionet_paths.append(file_path)
            else:
                print(f"File does not exist: {file_path}")
    return physionet_paths

test_paths = get_edf_paths(range(1, 80), OPEN_CLOSE_LEFT_RIGHT_FIST)
print(f"Found {len(test_paths)} EDF files")

if len(test_paths) == 0:
    raise ValueError("No EDF files found.")

parts = []
for path in test_paths:
    try:
        raw = mne.io.read_raw_edf(path, preload=True, stim_channel='auto', verbose='WARNING')
        sfreq = raw.info['sfreq']
        print(f"Sampling rate for {path}: {sfreq} Hz")
        if sfreq != 160:
            print(f"Resampling {path} from {sfreq} Hz to 160 Hz")
            raw.resample(160)
        parts.append(raw)
    except Exception as e:
        print(f"Error loading {path}: {str(e)}")

if len(parts) == 0:
    raise ValueError("No EDF files were successfully loaded.")

raw = mne.concatenate_raws(parts)
events, annot = mne.events_from_annotations(raw)
print(f"Annotations found: {list(annot.keys())}")
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
EEG_CHANNEL = len(eeg_channel_inds)
print(f"Number of EEG channels: {EEG_CHANNEL}")

# Create epochs
epoched = mne.Epochs(
    raw, events, dict(left=2, right=3), tmin=1, tmax=4.1,
    proj=False, picks=eeg_channel_inds, baseline=None, preload=True, verbose=True
)
X_test = epoched.get_data() * 1e3  # Convert to mV
y_test = epoched.events[:, 2] - 2  # Labels: 0=left, 1=right

# Resample to match training shape
expected_time_samples = int(4.1 * 160)  # 656 samples
current_time_samples = X_test.shape[-1]
print(f"Original test X shape: {X_test.shape}, Test y shape: {y_test.shape}")
if current_time_samples != expected_time_samples:
    print(f"Resampling epochs from {current_time_samples} to {expected_time_samples} samples")
    X_test_resampled = np.zeros((X_test.shape[0], X_test.shape[1], expected_time_samples), dtype=np.float32)
    for i in range(X_test.shape[0]):
        for j in range(X_test.shape[1]):
            X_test_resampled[i, j] = signal.resample(X_test[i, j], expected_time_samples)
    X_test = X_test_resampled
    print(f"Resampled test X shape: {X_test.shape}")

X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.int64)
print(f"Final test X shape: {X_test.shape}, Test y shape: {y_test.shape}")

# Class distribution
left_count = np.sum(y_test == 0)
right_count = np.sum(y_test == 1)
print(f"Class distribution: Left={left_count}, Right={right_count}")

Loading test EDF files...
Found 237 EDF files
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S001/S001R03.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S001/S001R07.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S001/S001R11.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S002/S002R03.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S002/S002R07.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S002/S002R11.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S003/S003R03.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S003/S003R07.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S003/S003R11.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dataset/raw_data/files/S004/S004R03.edf: 160.0 Hz
Sampling rate for /content/drive/MyDrive/dat

In [ ]:
class EEGDataset(data.Dataset):
    def __init__(self, x, y=None, inference=False):
        super().__init__()
        self.__split = "test" if not inference else "inference"
        self.dataset = {'x': x, 'y': y} if not inference else {'x': x}

    def __len__(self):
        return len(self.dataset['x'])

    def __getitem__(self, idx):
        x = self.dataset['x'][idx]
        x = torch.tensor(x).float()
        if self.__split != "inference":
            y = self.dataset['y'][idx]
            y = torch.tensor(y).unsqueeze(-1).float()
            return x, y
        return x

test_dataset = EEGDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
def evaluate_model(model, test_loader, model_type, eeg_channel):
    accuracy = Accuracy(task="binary")
    precision = Precision(task="binary", average='macro', num_classes=2)
    recall = Recall(task="binary", average='macro', num_classes=2)
    f1 = F1Score(task="binary", average='macro', num_classes=2)
    predictions = []
    probabilities = []
    true_labels = []
    test_loss = []
    running_accuracy = []
    cumulative_correct = 0
    total_samples = 0

    with torch.no_grad():
        for x, y in test_loader:
            y_hat = model(x)
            prob = torch.sigmoid(y_hat)
            pred = (prob > 0.5).float()
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            test_loss.append(loss.item())
            predictions.append(pred.cpu().numpy())
            probabilities.append(prob.cpu().numpy())
            true_labels.append(y.cpu().numpy())
            accuracy.update(y_hat, y)
            precision.update(y_hat, y)
            recall.update(y_hat, y)
            f1.update(y_hat, y)
            cumulative_correct += (pred == y).float().sum().item()
            total_samples += y.size(0)
            running_accuracy.append(cumulative_correct / total_samples)

    predictions = np.concatenate(predictions).flatten()
    probabilities = np.concatenate(probabilities).flatten()
    true_labels = np.concatenate(true_labels).flatten()
    avg_loss = np.mean(test_loss)
    acc = accuracy.compute().item()
    prec = precision.compute().item()
    rec = recall.compute().item()
    f1_score = f1.compute().item()

    # Per-class metrics
    correct_left = np.sum((predictions == 0) & (true_labels == 0))
    correct_right = np.sum((predictions == 1) & (true_labels == 1))
    total_left = np.sum(true_labels == 0)
    total_right = np.sum(true_labels == 1)

    # Confusion matrix
    cm = confusion_matrix(true_labels, predictions)

    # ROC and AUC
    fpr, tpr, _ = roc_curve(true_labels, probabilities)
    roc_auc = auc(fpr, tpr)

    # Visualizations
    # Separate scatterplots for each class
    left_indices = np.where(true_labels == 0)[0][:100]
    right_indices = np.where(true_labels == 1)[0][:100]

    plt.figure(figsize=(10, 5))
    correct_mask_left = predictions[left_indices] == true_labels[left_indices]
    plt.scatter(left_indices, probabilities[left_indices], c=np.where(correct_mask_left, '#33CAFF', '#2EBA15'), alpha=0.6)
    plt.axhline(y=0.5, color='r', linestyle='--', label='Threshold (0.5)')
    plt.title(f"{model_type} Model: Predicted Probabilities for Left Class (First 100)")
    plt.xlabel("Sample Index")
    plt.ylabel("Predicted Probability")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f"/content/{model_type}_left_scatter.png")
    plt.clf()

    plt.figure(figsize=(10, 5))
    correct_mask_right = predictions[right_indices] == true_labels[right_indices]
    plt.scatter(right_indices, probabilities[right_indices], c=np.where(correct_mask_right, '#33CAFF', '#2EBA15'), alpha=0.6)
    plt.axhline(y=0.5, color='r', linestyle='--', label='Threshold (0.5)')
    plt.title(f"{model_type} Model: Predicted Probabilities for Right Class (First 100)")
    plt.xlabel("Sample Index")
    plt.ylabel("Predicted Probability")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f"/content/{model_type}_right_scatter.png")
    plt.clf()

    # Accuracy bar plot
    class_accuracy = [correct_left/total_left if total_left > 0 else 0, correct_right/total_right if total_right > 0 else 0]
    plt.figure(figsize=(8, 5))
    plt.bar(CLASSES, class_accuracy, color=['#87CEEB', '#90EE90'])
    plt.title(f"{model_type} Model: Accuracy per Class")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    for i, v in enumerate(class_accuracy):
        plt.text(i, v + 0.02, f"{v:.4f}", ha='center')
    plt.grid(True, alpha=0.3)
    plt.savefig(f"/content/{model_type}_class_accuracy.png")
    plt.clf()

    # Running accuracy curve
    plt.figure(figsize=(10, 5))
    plt.plot(running_accuracy, color='#33CAFF', label='Test Accuracy')
    plt.title(f"{model_type} Model: Test Accuracy Curve")
    plt.xlabel("Sample Index")
    plt.ylabel("Cumulative Accuracy")
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f"/content/{model_type}_accuracy_curve.png")
    plt.clf()

    # Confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
    plt.title(f"{model_type} Model: Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(f"/content/{model_type}_confusion_matrix.png")
    plt.clf()

    # ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{model_type} Model: ROC Curve")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f"/content/{model_type}_roc_curve.png")
    plt.clf()

    # Save predictions to CSV
    pred_df = pd.DataFrame({
        'Sample Index': range(len(true_labels)),
        'True Label': true_labels,
        'Predicted Label': predictions,
        'Probability': probabilities
    })
    pred_df.to_csv(f"/content/{model_type}_predictions.csv", index=False)

    return {
        'loss': avg_loss,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1_score': f1_score,
        'correct_left': correct_left,
        'total_left': total_left,
        'correct_right': correct_right,
        'total_right': total_right,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'predictions_csv': f"/content/{model_type}_predictions.csv",
        'visualizations': [
            f"/content/{model_type}_left_scatter.png",
            f"/content/{model_type}_right_scatter.png",
            f"/content/{model_type}_class_accuracy.png",
            f"/content/{model_type}_accuracy_curve.png",
            f"/content/{model_type}_confusion_matrix.png",
            f"/content/{model_type}_roc_curve.png"
        ]
    }

In [ ]:
print("Testing PyTorch Model...")
model_pytorch = EEGClassificationModel(eeg_channel=EEG_CHANNEL, dropout=0.125)
model_pytorch.load_state_dict(torch.load(os.path.join(MODEL_DIR, "model_pytorch.pth")))
model_pytorch.eval()
pytorch_results = evaluate_model(model_pytorch, test_loader, "PyTorch", EEG_CHANNEL)

Testing PyTorch Model...


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [ ]:
class ModelWrapper(pl.LightningModule):
    def __init__(self, arch):
        super().__init__()
        self.arch = arch
        self.test_accuracy = Accuracy(task="binary")
        self.test_precision = Precision(task="binary", average='macro', num_classes=2)
        self.test_recall = Recall(task="binary", average='macro', num_classes=2)
        self.test_f1 = F1Score(task="binary", average='macro', num_classes=2)

    def forward(self, x):
        return self.arch(x)

    def test_step(self, batch, batch_nb):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        self.test_accuracy.update(y_hat, y)
        self.test_precision.update(y_hat, y)
        self.test_recall.update(y_hat, y)
        self.test_f1.update(y_hat, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy.compute(), prog_bar=True)
        self.log("test_precision", self.test_precision.compute(), prog_bar=True)
        self.log("test_recall", self.test_recall.compute(), prog_bar=True)
        self.log("test_f1", self.test_f1.compute(), prog_bar=True)

print("Testing Lightning Model...")
model_lightning = EEGClassificationModel(eeg_channel=EEG_CHANNEL, dropout=0.125)
wrapper = ModelWrapper(model_lightning)
checkpoint = torch.load(os.path.join(MODEL_DIR, "model_lightning.ckpt"))
wrapper.load_state_dict(checkpoint['state_dict'])
wrapper.eval()
trainer = pl.Trainer(accelerator="auto", devices=1)
trainer.test(wrapper, dataloaders=test_loader)
lightning_results = {
    'loss': trainer.logged_metrics.get('test_loss', 0.0),
    'accuracy': trainer.logged_metrics.get('test_acc', 0.0),
    'precision': trainer.logged_metrics.get('test_precision', 0.0),
    'recall': trainer.logged_metrics.get('test_recall', 0.0),
    'f1_score': trainer.logged_metrics.get('test_f1', 0.0)
}

Testing Lightning Model...


INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7059139609336853     │
│          test_f1          │    0.6898722052574158     │
│         test_loss         │    0.8122265338897705     │
│      test_precision       │    0.7276922464370728     │
│        test_recall        │    0.6560699939727783     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
lightning_results.update(evaluate_model(wrapper, test_loader, "Lightning", EEG_CHANNEL))

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [ ]:
print("Testing ONNX Model...")
ort_session = ort.InferenceSession(os.path.join(MODEL_DIR, "model.onnx"))
def onnx_model(x):
    x_np = x.numpy()
    expected_shape = (1, EEG_CHANNEL, int(4.1 * 160))
    if x_np.shape != expected_shape:
        raise ValueError(f"Input shape {x_np.shape} does not match expected {expected_shape}")
    ort_inputs = {ort_session.get_inputs()[0].name: x_np}
    y_hat = torch.tensor(ort_session.run(None, ort_inputs)[0])
    return y_hat

onnx_results = evaluate_model(onnx_model, test_loader, "ONNX", EEG_CHANNEL)

Testing ONNX Model...


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [ ]:
print("\n=== Test Results Summary ===")
print(f"Class Distribution: Left={left_count}, Right={right_count}")
for model_type, results in [("PyTorch", pytorch_results), ("Lightning", lightning_results), ("ONNX", onnx_results)]:
    print(f"\n{model_type} Model:")
    print(f"  Test Loss: {results['loss']:.4f}")
    print(f"  Accuracy: {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1-Score: {results['f1_score']:.4f}")
    print(f"  Left Class: {results['correct_left']}/{results['total_left']} (Accuracy: {results['correct_left']/results['total_left']:.4f})")
    print(f"  Right Class: {results['correct_right']}/{results['total_right']} (Accuracy: {results['correct_right']/results['total_right']:.4f})")
    print(f"  ROC AUC: {results['roc_auc']:.4f}")
    print(f"  Confusion Matrix:\n{results['confusion_matrix']}")
    print(f"  Predictions saved to: {results['predictions_csv']}")
    print(f"  Visualizations saved to: {results['visualizations']}")


=== Test Results Summary ===
Class Distribution: Left=1688, Right=1678

PyTorch Model:
  Test Loss: 0.8122
  Accuracy: 0.7077
  Precision: 0.7265
  Recall: 0.6633
  F1-Score: 0.6935
  Left Class: 1141/1688 (Accuracy: 0.6759)
  Right Class: 1230/1678 (Accuracy: 0.7330)
  ROC AUC: 0.7679
  Confusion Matrix:
[[1141  547]
 [ 448 1230]]
  Predictions saved to: /content/PyTorch_predictions.csv
  Visualizations saved to: ['/content/PyTorch_left_scatter.png', '/content/PyTorch_right_scatter.png', '/content/PyTorch_class_accuracy.png', '/content/PyTorch_accuracy_curve.png', '/content/PyTorch_confusion_matrix.png', '/content/PyTorch_roc_curve.png']

Lightning Model:
  Test Loss: 0.8122
  Accuracy: 0.7077
  Precision: 0.7265
  Recall: 0.6633
  F1-Score: 0.6935
  Left Class: 1141/1688 (Accuracy: 0.6759)
  Right Class: 1230/1678 (Accuracy: 0.7330)
  ROC AUC: 0.7679
  Confusion Matrix:
[[1141  547]
 [ 448 1230]]
  Predictions saved to: /content/Lightning_predictions.csv
  Visualizations saved to: [